In [1]:
from torch.nn.modules.activation import ReLU

import torch
import torch.nn as nn

In [2]:
def conv_bn_relu_pool(in_channels, out_channels, kernel_size=2, stride=1, padding=0):
  return nn.Sequential(
         nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding, bias=False),
         nn.BatchNorm2d(out_channels),
         nn.ReLU(inplace=True),
         nn.MaxPool2d(kernel_size=kernel_size, stride=2, padding=0)
    )

In [3]:
def fc_bn_relu(in_features, out_features):
  return nn.Sequential(
      nn.Linear(in_features=in_features, out_features=out_features, bias=False),
      nn.BatchNorm1d(out_features),
      nn.ReLU(inplace=True)
  )

In [4]:
class my_model(nn.Module):
    def __init__(self):
        ## 여기에 코드 작성
        super().__init__()
        
        self.expand_channels = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True)
        )
        
        self.conv_bn_relu = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=2, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True))
        
        self.conv_bn_relu_pool1 = conv_bn_relu_pool(64, 128, kernel_size=2, stride=1, padding=0)
        self.conv_bn_relu_pool2 = conv_bn_relu_pool(128, 256, kernel_size=2, stride=1, padding=0)
        self.conv_bn_relu_pool3 = conv_bn_relu_pool(256, 512, kernel_size=2, stride=1, padding=0)
        self.conv_bn_relu_pool4 = conv_bn_relu_pool(512, 1024, kernel_size=2, stride=1, padding=0)

        self.flatten = nn.Flatten()

        self.fc_bn_relu1 = fc_bn_relu(in_features=1024*2*2, out_features=512)
        self.fc_bn_relu2 = fc_bn_relu(in_features=512, out_features=512)
        self.fc_bn_relu3 = fc_bn_relu(in_features=512, out_features=7)


    def forward(self, x):
        ## 여기에 코드 작성
        x = self.expand_channels(x)
        x = self.conv_bn_relu(x)
        x = self.conv_bn_relu_pool1(x)
        x = self.conv_bn_relu_pool2(x)
        x = self.conv_bn_relu_pool3(x)
        x = self.conv_bn_relu_pool4(x)
        x = self.flatten(x)
        x = self.fc_bn_relu1(x)
        x = self.fc_bn_relu2(x)
        x = self.fc_bn_relu3(x)

        return x

In [5]:
if __name__ == '__main__':
    x = torch.randn((2, 1, 48, 48))
    model = my_model()
    y = model(x)
    #print(y.squeeze().shape)